In [1]:
import os
import sys
sys.path.append(os.getcwd())
import random
from sklearn.model_selection import train_test_split
import numpy as np
import pickle as pkl
import open3d as o3d
from dataset.sample_points import SampleMethod
from models.utils import save_pcd
from collections import Counter
import argparse
import zipfile


In [2]:
def convert_to_mesh(instance_dir, sample_strategy, simplify=False, output_dir='./data/shapenet/mesh'):
    os.makedirs(output_dir, exist_ok=True)
    manifold_script_path = os.path.join(MANIFOLD_DIR, 'manifold')
    simplify_sciprt_path = os.path.join(MANIFOLD_DIR, 'simplify')
    for instance in instance_dir:
        prefix_name = instance.split('/')[-1]
        instance_path = os.path.join(instance, 'model.obj')
        output_path = os.path.join(output_dir, prefix_name + '.obj')
        if sample_strategy[prefix_name] == 'montecarlo_sample':
            if not simplify:
                # generate watertight mesh
                cmd = "{} {} {} {}".format(manifold_script_path, instance_path, output_path, 20000)
                print(cmd)
                os.system(cmd)
            else:
                # the input mesh should be watertight
                mesh = o3d.io.read_triangle_mesh(instance_path)
                assert mesh.is_edge_manifold() == True

                cmd = "{} -i {} -o {} -f {}".format(simplify_sciprt_path, instance_path, output_path, 40000)
                print(cmd)
                os.system(cmd)
        else:
            # directly use the cad model
            cad_model = o3d.io.read_triangle_mesh(instance_path)
            o3d.io.write_triangle_mesh(output_path, cad_model, write_vertex_colors=False, write_triangle_uvs=False)


In [3]:
def normalize_pcd(xyzs):
    '''
     normalize xyzs to [0,1], keep ratio unchanged
    '''
    shift = np.mean(xyzs, axis=0)
    xyzs -= shift
    max_coord, min_coord = np.max(xyzs), np.min(xyzs)
    xyzs = xyzs - min_coord
    xyzs = xyzs / (max_coord - min_coord)
    meta_data = {}
    meta_data['shift'] = shift
    meta_data['max_coord'] = max_coord
    meta_data['min_coord'] = min_coord
    return xyzs, meta_data


In [4]:
def extract_zip_files(directory):
    for item in os.listdir(directory):  # loop through items in dir
        if item.endswith('.zip'):  # check for ".zip" extension
            file_name = os.path.join(directory, item)  # get full path of the file
            print(f"Extracting {file_name}...")
            zip_ref = zipfile.ZipFile(file_name)  # create a ZipFile object
            zip_ref.extractall(directory)  # extract the file to the directory
            zip_ref.close()  # close the file
            print(f"Finished extracting {file_name}")

            print(f"Deleting {file_name}...")
            os.remove(file_name)  # remove the zip file
            print(f"Deleted {file_name}")
        else:
            print(f"Skipping {item}, not a .zip file")

In [5]:

def get_instance_dir(data_root, instance_num, max_samples=400):
    data_dir = [os.path.join(data_root, dir) for dir in os.listdir(data_root) if os.path.isdir(os.path.join(data_root, dir))]
    instance_dir = []
    for dir in data_dir:
        full_instance_name = os.listdir(dir)
        if len(full_instance_name) > instance_num:
            instance_name = random.sample(full_instance_name, instance_num)
        else:
            instance_name = full_instance_name
        instance_dir += [os.path.join(dir, name) for name in instance_name]
    return instance_dir[:max_samples]



In [6]:
def get_sample_strategy(instance_dir, split_rate):
    sample_strategy = {}
    montecarlo_sample_instance = set(random.sample(instance_dir, int(len(instance_dir)*split_rate)))
    for instance_path in instance_dir:
        instance_name = instance_path.split('/')[-1]
        if instance_path in montecarlo_sample_instance:
            cur_sample_strategy = 'montecarlo_sample'
        else:
            cur_sample_strategy = 'subdivide'
        sample_strategy[instance_name] = cur_sample_strategy
    return sample_strategy

In [7]:
def sample_pcd_from_mesh(mesh_dir, pcd_dir, sample_strategy):
    mesh_name = os.listdir(mesh_dir)
    os.makedirs(pcd_dir, exist_ok=True)
    for name in mesh_name:
        if not name.endswith('.obj'):
            continue
        prefix_name = os.path.splitext(name)[0]
        mesh_path = os.path.join(mesh_dir, name)
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        # current sample strategy
        cur_sample_strategy = sample_strategy[prefix_name]
        if cur_sample_strategy == 'montecarlo_sample':
            mesh.compute_vertex_normals(normalized=True)
            pcd = mesh.sample_points_uniformly(200000)
        else:
            subdivide_mesh = mesh.subdivide_midpoint(3)
            # the normals may be [0, 0, 0]
            subdivide_mesh.compute_vertex_normals(normalized=True)
            pcd = o3d.geometry.PointCloud()
            pcd.points = subdivide_mesh.vertices
            pcd.normals = subdivide_mesh.vertex_normals
        save_pcd(pcd_dir, prefix_name + '.ply', np.asarray(pcd.points), np.asarray(pcd.normals))
        # o3d.io.write_point_cloud(os.path.join(pcd_dir, prefix_name + '.ply'), pcd)



In [8]:
def generate_path_txt(pcd_dir, output_dir, sample_strategy):
    pcd_name = os.listdir(pcd_dir)
    montecarlo_sample_path = [os.path.join(pcd_dir, p)+' montecarlo_sample\n' for p in pcd_name if sample_strategy[os.path.splitext(p)[0]]=='montecarlo_sample']
    subdivide_path = [os.path.join(pcd_dir, p)+' subdivide\n' for p in pcd_name if sample_strategy[os.path.splitext(p)[0]]=='subdivide']
    # train: 7 test: 2 val: 1
    montecarlo_sample_train_path, montecarlo_sample_test_path = train_test_split(montecarlo_sample_path, test_size=0.3)
    montecarlo_sample_test_path, montecarlo_sample_val_path = train_test_split(montecarlo_sample_test_path, test_size=1/3)
    subdivide_train_path, subdivide_test_path = train_test_split(subdivide_path, test_size=0.3)
    subdivide_test_path, subdivide_val_path = train_test_split(subdivide_test_path, test_size=1/3)
    train_path = montecarlo_sample_train_path + subdivide_train_path
    test_path = montecarlo_sample_test_path + subdivide_test_path
    val_path = montecarlo_sample_val_path + subdivide_val_path
    # generate txt file
    modes = ['train', 'test', 'val']
    for mode in modes:
        with open(os.path.join(output_dir, mode + '.txt'), 'w')as f:
            f.writelines(eval(mode+'_path'))



In [9]:
def divide_cube(xyzs, attribute, map_size=100, cube_size=10, min_num=100, max_num=30000, cur_sample_strategy=None):
    '''
    points : N x 3
    resolution: 100 x 100 x 100
    cube_size: 10 x 10 x 10
    min_num and max_num points in each cube, if small than min_num or larger than max_num then discard it
    cur_sample_strategy: montecarlo_sample or subdivide
    return label that indicates each points' cube_idx
    '''
    assert cur_sample_strategy in ['montecarlo_sample', 'subdivide']
    Sample = SampleMethod(rate=0.7)

    output_points = {}
    xyzs , meta_data= normalize_pcd(xyzs)

    map_xyzs = xyzs * (map_size)
    xyzs = np.floor(map_xyzs).astype('float32')
    label = np.zeros(xyzs.shape[0]).astype(int) - 1

    cubes = {}
    for idx, p in enumerate(xyzs):
        tuple_cube_idx = tuple((p//cube_size).astype(int))
        if not tuple_cube_idx in cubes.keys():
            cubes[tuple_cube_idx] = []
        cubes[tuple_cube_idx].append(idx)

    # remove those cubes whose points_num is small than min_num
    del_k = -1
    k_del = []
    for k in cubes.keys():
        if len(cubes[k]) < min_num:
            label[cubes[k]] = del_k
            del_k -= 1
            k_del.append(k)
        if len(cubes[k]) > max_num:
            label[cubes[k]] = del_k
            del_k -= 1
            k_del.append(k)
    for k in k_del:
        del cubes[k]

    double_check = []
    for idx, p in cubes.items():
        if len(p) < min_num:
            # print('del ', len(p))
            double_check.append(idx)

    for i in double_check:
        del cubes[i]

    for tuple_cube_idx, point_idx in cubes.items():
        dim_cube_num = np.ceil(map_size/cube_size).astype(int)
        # indicate which cube a point belongs to
        cube_idx = tuple_cube_idx[0] * dim_cube_num * dim_cube_num + \
                  tuple_cube_idx [1] * dim_cube_num + tuple_cube_idx[2]
        label[point_idx] = cube_idx

        if cur_sample_strategy == 'montecarlo_sample':
            tmp_p, tmp_normal = map_xyzs[point_idx, :], attribute[point_idx, :]
            if tmp_p.shape[0] < min_num:
                print(tmp_p.shape)
                raise ValueError
            # dim = new_points.shape[-1]
            final_points = Sample.random_sample(tmp_p, tmp_normal, min_point=min_num)
            output_points[cube_idx] = final_points
            if final_points.shape[0] < min_num:
                print(final_points.shape)
                raise ValueError
        else:
            output_points[cube_idx] = np.concatenate([map_xyzs[point_idx, :], attribute[point_idx, :]], -1)

    # label indicates each points' cube index
    # label may have some values less than 0, which should be ignored
    num = [len(k)<min_num for k in output_points.values()]
    if sum(num) > 0:
        print(sorted([len(k) for k in output_points.values()]))
        raise ValueError

    return label, output_points, meta_data


In [10]:
def generate_dataset(mode='train', dataset_name='shapenet', cube_size=20, min_num=100, max_num=100000, save_path='./data/shapenet'):
    path_txt = os.path.join(save_path, f'{mode}.txt')
    with open(path_txt, 'r')as f:
        pcd_item = f.readlines()

    data = {}
    invalid_path =[]
    idx = 0
    patch_num = 0
    for item in pcd_item:
        path, cur_sample_strategy = item.strip('\n').split(' ')

        pcd = o3d.io.read_point_cloud(path)
        xyzs = np.array(pcd.points)
        normals = np.array(pcd.normals)
        
        if xyzs.size == 0:
            print(f"Warning: Empty point cloud for file {path}")
            continue

        mask, points, meta_data = divide_cube(xyzs, attribute=normals, cube_size=cube_size, min_num=min_num,
                                              max_num=max_num, cur_sample_strategy=cur_sample_strategy)
        key = Counter(mask)
        # print(len(np.unique(mask)))
        print('----------------')
        print('valid patch number:', len(points))
        k = np.array(list(key.values()))
        if len(points) == 0:
            print(20 * '***')
            invalid_path.append(path)
            continue

        data[idx] = {}
        data[idx]['points'] = points
        data[idx]['meta_data'] = meta_data
        idx += 1
        patch_num += len(points)

    with open(os.path.join(save_path, f'{dataset_name}_{mode}_cube_size_{cube_size}.pkl'), 'wb')as f:
        pkl.dump(data, f, protocol=2)


In [11]:
class Args:
    def __init__(self):
        # data root
        self.data_root = './'  # Set your path here
        # instance number of each class
        self.instance_num = 50
        # split rate for montecarlo sampling and subdivision
        self.split_rate = 0.8
        # save dir for watertight mesh
        self.output_mesh_dir = './data/shapenet/mesh'
        # save dir for sampled point cloud
        self.output_pcd_dir = './data/shapenet/pcd'
        # cube size
        self.cube_size = 22
        # minimum points number in each cube when training
        self.train_min_num = 1024
        # minimum points number in each cube when testing
        self.test_min_num = 100
        # maximum points number in each cube
        self.max_num = 500000

def parse_dataset_args():
    return Args()


In [12]:
MANIFOLD_DIR = './dataset/Manifold/build/'

In [13]:
dataset_args = parse_dataset_args()

In [14]:
dataset_args.data_root = './ShapeNetCore.v1'

In [15]:
extract_zip_files(dataset_args.data_root)

Skipping 02691156, not a .zip file
Skipping 02691156.csv, not a .zip file
Skipping 02747177, not a .zip file
Skipping 02747177.csv, not a .zip file
Skipping 02773838, not a .zip file
Skipping 02773838.csv, not a .zip file
Skipping 02801938, not a .zip file
Skipping 02801938.csv, not a .zip file
Skipping 02808440, not a .zip file
Skipping 02808440.csv, not a .zip file
Skipping 02818832, not a .zip file
Skipping 02818832.csv, not a .zip file
Skipping 02828884, not a .zip file
Skipping 02828884.csv, not a .zip file
Skipping 02834778, not a .zip file
Skipping 02834778.csv, not a .zip file
Skipping 02843684, not a .zip file
Skipping 02843684.csv, not a .zip file
Skipping 02858304.csv, not a .zip file
Skipping 02871439, not a .zip file
Skipping 02871439.csv, not a .zip file
Skipping 02876657, not a .zip file
Skipping 02876657.csv, not a .zip file
Skipping 02880940, not a .zip file
Skipping 02880940.csv, not a .zip file
Skipping 02924116, not a .zip file
Skipping 02924116.csv, not a .zip file

In [16]:
instance_dir = get_instance_dir(dataset_args.data_root, instance_num=dataset_args.instance_num, max_samples=100)
instance_dir

['./ShapeNetCore.v1/02691156/f9c93ddc9b921d88a72ae0d8b7f98590',
 './ShapeNetCore.v1/02691156/b590adb6d3486f6e90b1d6deb98feec6',
 './ShapeNetCore.v1/02691156/b4a420a55d3db8aca89fa467f217f46',
 './ShapeNetCore.v1/02691156/d54a694d514b1911844ac48bcfce34',
 './ShapeNetCore.v1/02691156/4f9b12d07dce21ac9d93a50cb0355558',
 './ShapeNetCore.v1/02691156/df8bb967a346ccc25e9e2656aff7dd5b',
 './ShapeNetCore.v1/02691156/d01da87ecbc2deea27e33924ab17ba05',
 './ShapeNetCore.v1/02691156/4ee48907120e261c3df16180af9974ee',
 './ShapeNetCore.v1/02691156/404714cf775f5ceed32761b337f8b72a',
 './ShapeNetCore.v1/02691156/106dfe858cb8fbc2afc6b80d80a265ab',
 './ShapeNetCore.v1/02691156/56ba815f883279b462b600da24e0965',
 './ShapeNetCore.v1/02691156/3572e3fa25911295922e63c830b776de',
 './ShapeNetCore.v1/02691156/78f32f279a50beea216b327569dd806e',
 './ShapeNetCore.v1/02691156/dd9a7dd5e2ea3389938204d34a891739',
 './ShapeNetCore.v1/02691156/987e67ef16969b93663e90eaf6b4ca52',
 './ShapeNetCore.v1/02691156/6d119be1286c7b1

In [18]:
sample_strategy = get_sample_strategy(instance_dir, dataset_args.split_rate)

In [19]:
convert_to_mesh(instance_dir, sample_strategy, simplify=False, output_dir=dataset_args.output_mesh_dir)

./dataset/Manifold/build/manifold ./ShapeNetCore.v1/02691156/d54a694d514b1911844ac48bcfce34/model.obj ./data/shapenet/mesh/d54a694d514b1911844ac48bcfce34.obj 20000


  o mesh14_mesh14-geometry
  o mesh13_mesh13-geometry
  o mesh12_mesh12-geometry
  o mesh11_mesh11-geometry
  o mesh10_mesh10-geometry
  o mesh9_mesh9-geometry
  o mesh8_mesh8-geometry
  o mesh7_mesh7-geometry
  o mesh6_mesh6-geometry
  o mesh5.003_mesh5-geometry
  o mesh4.003_mesh4-geometry
  o mesh3.003_mesh3-geometry
  o mesh2.096_mesh2-geometry
  o mesh1.096_mesh1-geometry
  o mesh2.095_mesh2-geometry
  o mesh1.095_mesh1-geometry
  o mesh2.094_mesh2-geometry
  o mesh1.094_mesh1-geometry
  o mesh2.093_mesh2-geometry
  o mesh1.093_mesh1-geometry
  o mesh2.092_mesh2-geometry
  o mesh1.092_mesh1-geometry
  o mesh2.091_mesh2-geometry
  o mesh1.091_mesh1-geometry
  o mesh2.090_mesh2-geometry
  o mesh1.090_mesh1-geometry
  o mesh2.089_mesh2-geometry
  o mesh1.089_mesh1-geometry
  o mesh2.088_mesh2-geometry
  o mesh1.088_mesh1-geometry
  o mesh2.087_mesh2-geometry
  o mesh1.087_mesh1-geometry
  o mesh2.086_mesh2-geometry
  o mesh1.086_mesh1-geometry
  o mesh2.085_mesh2-geometry
  o mesh1.0

manifold ./ShapeNetCore.v1/02691156/d54a694d514b1911844ac48bcfce34/model.obj ./data/shapenet/mesh/d54a694d514b1911844ac48bcfce34.obj 20000
./dataset/Manifold/build/manifold ./ShapeNetCore.v1/02691156/4f9b12d07dce21ac9d93a50cb0355558/model.obj ./data/shapenet/mesh/4f9b12d07dce21ac9d93a50cb0355558.obj 20000


  o mesh27_mesh27-geometry
  o mesh26_mesh26-geometry
  o mesh25_mesh25-geometry
  o mesh24_mesh24-geometry
  o mesh23_mesh23-geometry
  o mesh22_mesh22-geometry
  o mesh21_mesh21-geometry
  o mesh20_mesh20-geometry
  o mesh19_mesh19-geometry
  o mesh18_mesh18-geometry
  o mesh17_mesh17-geometry
  o mesh16_mesh16-geometry
  o mesh15_mesh15-geometry
  o mesh14_mesh14-geometry
  o mesh13_mesh13-geometry
  o mesh12_mesh12-geometry
  o mesh11_mesh11-geometry
  o mesh10_mesh10-geometry
  o mesh9_mesh9-geometry
  o mesh8_mesh8-geometry
  o mesh7_mesh7-geometry
  o mesh6_mesh6-geometry
  o mesh5_mesh5-geometry
  o mesh4_mesh4-geometry
  o mesh3_mesh3-geometry
  o mesh2_mesh2-geometry
  o mesh1_mesh1-geometry


manifold ./ShapeNetCore.v1/02691156/4f9b12d07dce21ac9d93a50cb0355558/model.obj ./data/shapenet/mesh/4f9b12d07dce21ac9d93a50cb0355558.obj 20000
./dataset/Manifold/build/manifold ./ShapeNetCore.v1/02691156/d01da87ecbc2deea27e33924ab17ba05/model.obj ./data/shapenet/mesh/d01da87ecbc2deea27e33924ab17ba05.obj 20000


...

manifold ./ShapeNetCore.v1/02747177/7919df48b23e61ba6739a7caa0c577bd/model.obj ./data/shapenet/mesh/7919df48b23e61ba6739a7caa0c577bd.obj 20000
./dataset/Manifold/build/manifold ./ShapeNetCore.v1/02747177/e5b7b5112e396a49b581cc9c4410f841/model.obj ./data/shapenet/mesh/e5b7b5112e396a49b581cc9c4410f841.obj 20000


  o mesh5.001_mesh5-geometry
  o mesh4.001_mesh4-geometry
  o mesh3.001_mesh3-geometry
  o mesh2.001_mesh2-geometry
  o mesh1.001_mesh1-geometry


manifold ./ShapeNetCore.v1/02747177/e5b7b5112e396a49b581cc9c4410f841/model.obj ./data/shapenet/mesh/e5b7b5112e396a49b581cc9c4410f841.obj 20000
./dataset/Manifold/build/manifold ./ShapeNetCore.v1/02747177/a9cd539f8a5e19e06af350e51093426c/model.obj ./data/shapenet/mesh/a9cd539f8a5e19e06af350e51093426c.obj 20000


  o mesh1_mesh1-geometry


manifold ./ShapeNetCore.v1/02747177/a9cd539f8a5e19e06af350e51093426c/model.obj ./data/shapenet/mesh/a9cd539f8a5e19e06af350e51093426c.obj 20000


In [20]:
sample_pcd_from_mesh(dataset_args.output_mesh_dir, dataset_args.output_pcd_dir, sample_strategy)

In [21]:
generate_path_txt(dataset_args.output_pcd_dir, './data/shapenet', sample_strategy)

In [22]:
generate_dataset('train', 'shapenet', cube_size=dataset_args.cube_size, min_num=dataset_args.train_min_num,
                     max_num=dataset_args.max_num, save_path='./data/shapenet')

----------------
valid patch number: 44
----------------
valid patch number: 55
----------------
valid patch number: 23
----------------
valid patch number: 53
----------------
valid patch number: 19
----------------
valid patch number: 38
----------------
valid patch number: 42
----------------
valid patch number: 21
----------------
valid patch number: 64
----------------
valid patch number: 56
----------------
valid patch number: 47
----------------
valid patch number: 22
----------------
valid patch number: 39
----------------
valid patch number: 43
----------------
valid patch number: 18
----------------
valid patch number: 56
----------------
valid patch number: 23
----------------
valid patch number: 59
----------------
valid patch number: 66
----------------
valid patch number: 14
----------------
valid patch number: 50
----------------
valid patch number: 20
----------------
valid patch number: 17
----------------
valid patch number: 15
----------------
valid patch number: 78


In [23]:
generate_dataset('val', 'shapenet', cube_size=dataset_args.cube_size, min_num=dataset_args.train_min_num,
                     max_num=dataset_args.max_num, save_path='./data/shapenet')

----------------
valid patch number: 42
----------------
valid patch number: 24
----------------
valid patch number: 23
----------------
valid patch number: 42
----------------
valid patch number: 38
----------------
valid patch number: 20
----------------
valid patch number: 19
----------------
valid patch number: 25
----------------
valid patch number: 20
----------------
valid patch number: 14


In [24]:
generate_dataset('test', 'shapenet', cube_size=dataset_args.cube_size, min_num=dataset_args.test_min_num,
                     max_num=dataset_args.max_num, save_path='./data/shapenet')

----------------
valid patch number: 12
----------------
valid patch number: 60
----------------
valid patch number: 42
----------------
valid patch number: 67
----------------
valid patch number: 72
----------------
valid patch number: 65
----------------
valid patch number: 19
----------------
valid patch number: 96
----------------
valid patch number: 102
----------------
valid patch number: 26
----------------
valid patch number: 16
----------------
valid patch number: 57
----------------
valid patch number: 24
----------------
valid patch number: 64
----------------
valid patch number: 26
----------------
valid patch number: 57
----------------
valid patch number: 61
----------------
valid patch number: 46
----------------
valid patch number: 22
----------------
valid patch number: 16
